# This jupyter notebook is for assistant maintenance purposes only.

In [34]:
# %pip install --upgrade openai

In [35]:
#import the necessary libraries
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [36]:
client = OpenAI()

# Create the assistant with file_search enabled

Our first step is to create an Assistant that can do file searching regardless of where the vector store resides (Assistant or Thread)

In [42]:
#Creating the Assistant

assistant = client.beta.assistants.create(
    model="gpt-4o-mini",
    
    instructions="You are BisonGPT, a knowledgeable and supportive assistant dedicated to helping students at Howard University. You have access to a wide range of information about Howard, stored in various documents and files, which you can search to provide specific, accurate answers.\n\nYour main role is to assist students by answering questions, offering academic advice, and helping them navigate university processes. You can:\n - Recommend course schedules tailored to students' academic plans and graduation requirements.\n - Advise on graduation plans and pathways, based on program-specific requirements and deadlines.\n - Explain university policies, including financial aid, academic advising, and admissions.\n - Guide students on campus resources, student organizations, and events.\n\nWhen responding, aim to be clear, concise, and supportive. Reference the university's guidelines and the most relevant information available in your files to ensure your answers are both accurate and useful.",
    
    name="BisonGPT",
    
    tools=[
        {
            "type": "file_search"
        },
        {
            "type": "function",
            "function": {
                "name": "get_directions",
                "description": "Get directions to a given location.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "address": {
                            "type": "string",
                            "description": "An address e.g. 7501 Muirkirk Road, Beltsville, MD"
                        }
                    },
                    "required": ["address"]
                }
            }
        }
    ],
)

# Or retrieve the assistant

In [9]:
def retrieve_assistant(name):
    assistants = client.beta.assistants.list()
    for assistant in assistants:
        if assistant.name == name:
            return assistant
        
assistant = retrieve_assistant("BisonGPT")

# Create a vector store


In [6]:
vector_store = client.beta.vector_stores.create(name="Howard University Data")

# Or Retrieve the vector store

In [38]:
def retrieve_vector_store(name: str):

    vector_stores = client.beta.vector_stores.list()

    for vector_store in vector_stores:
        if vector_store.name == name:
            return vector_store

vector_store = retrieve_vector_store("Howard University Data")

## Add all files...

In [26]:
def get_all_files(directory_path):
    file_paths = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

def upload_files(files):
    file_streams = [open(path, "rb") for path in files]

    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id, files=file_streams
    )

    print(file_batch.status)
    print(file_batch.file_counts)

    for file_stream in file_streams:
        file_stream.close()


## Or check if any local files are not in the vector store

In [ ]:
files_to_add = []

def get_vector_store_files():
    files = []

    response = client.beta.vector_stores.files.list(vector_store_id=vector_store.id)

    for f in response.data:
        file = client.files.retrieve(f.id)
        files.append(file)

    return files

all_files = get_all_files("../Data/")

vector_store_files = get_vector_store_files()

vector_store_names = [file.filename for file in vector_store_files]

for path in all_files:
    filename = os.path.basename(path)
    
    if filename not in vector_store_names:
        files_to_add.append(path)

print(files_to_add)

upload_files(files_to_add)



['../Data/University\\buildings.json']


# Attach the vector store to the assisstant

In [45]:
try:
    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    )
except Exception as e:
    print(f"Failed to attach vector store to assistant: {e}")